In [1]:
pip install -U scikit-learn

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("output.csv")

In [4]:
data['Sales order date'] = pd.to_datetime(data['Sales order date'],dayfirst = True)
data['Quote Date']= pd.to_datetime(data['Quote Date'],dayfirst = True)
data['Quote Date - So Date Difference'] = (data['Sales order date'] - data['Quote Date']).dt.days
data['Date'] = pd.to_datetime(data['Date'],dayfirst = True)
data['Age of Customer(days)'] = (data['Date'] - data['Sales order date']).dt.days
data['Age of Customer(days)']=data['Age of Customer(days)'].fillna(0)

In [5]:
data['Conversion Ratio'] = 0.0
for i in data['Customer ID']:
    a=list(data.loc[data['Customer ID']==i,'Quote Amount'])
    l = list(data.loc[data['Customer ID']==i,'Sales Order Amount'])
    s = 0
    c= 0
    for j in l:
       
        if pd.isnull(j)==False:
            s = s+j
            c = c+1
    if c==0:
        m = 0 
    else:
        m = s
            
    d=sum(a)/len(a)
    data.loc[data['Customer ID']==i,'TotalSales']= m   
    rate=c/len(a)
    data.loc[data['Customer ID']==i,'Conversion Ratio']= round((rate*100),2)

In [6]:
for i in data['Customer ID']:
    a=list(data.loc[data['Customer ID']==i,'Age of Customer(days)'].values)
    data.loc[data['Customer ID']==i,'Age of Customer(days)']=(max(a))

In [7]:
data['Avg Days'] = 0.0
for i in data['Customer ID']:
    b = list(data.loc[data['Customer ID']==i,'Quote Date - So Date Difference'])
    s = 0
    c= 0
    for j in b:
        if pd.isnull(j)==False:
            s = s+j
            c = c+1
    if c==0:
        m = 0 
    else:
        m = s/c
            
    data.loc[data['Customer ID']==i,'Avg Days']= m

In [8]:
data['Age of Customer (years)'] = round((data['Age of Customer(days)']/365),2)

In [9]:
data2 = data.drop_duplicates(subset=['Customer ID'],keep = 'last')

In [10]:
pip install sklearn

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [11]:
from sklearn.preprocessing import MinMaxScaler
data2[['Conversion Ratio','Age of Customer (years)','TotalSales','Avg Days']] = MinMaxScaler().fit(data2[['Conversion Ratio','Age of Customer (years)','TotalSales','Avg Days']]).transform(data2[['Conversion Ratio','Age of Customer (years)','TotalSales','Avg Days']])  

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:3673: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [12]:
data2['Total Rating'] = data2['Conversion Ratio'] + (0.75 *data2['TotalSales']) +(0.2 * data2['Age of Customer (years)']) +(0.5 * data2['Avg Days'])

/tmp/ipykernel_662/1117038575.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Total Rating'] = data2['Conversion Ratio'] + (0.75 *data2['TotalSales']) +(0.2 * data2['Age of Customer (years)']) +(0.5 * data2['Avg Days'])


In [13]:
data2 = data2.sort_values(['Total Rating','Conversion Ratio','Age of Customer (years)','Avg Days'],ascending = [False,False,False,False])
data2 = data2.reset_index()
data2.drop(columns = ['index'],inplace = True)
data2 ['Rank']= data2.index+1

In [14]:
data2.to_csv("output.csv",index=False)